Como primer paso se configura una base de datos Postgres en Docker con el siguiente DDL empleando Dbeaver:

-- Tabla: Usuarios
CREATE TABLE Usuarios (
    UsuarioID SERIAL PRIMARY KEY,
    Nombre VARCHAR(100) NOT NULL,
    Apellido VARCHAR(100) NOT NULL,
    DNI VARCHAR(20) UNIQUE NOT NULL,
    Email VARCHAR(255) UNIQUE NOT NULL,
    Contraseña VARCHAR(255) NOT NULL,
    FechaRegistro TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

-- Tabla: Categorías
CREATE TABLE Categorias (
    CategoriaID SERIAL PRIMARY KEY,
    Nombre VARCHAR(100) NOT NULL,
    Descripcion VARCHAR(255)
);

-- Tabla: Productos
CREATE TABLE Productos (
    ProductoID SERIAL PRIMARY KEY,
    Nombre VARCHAR(255) NOT NULL,
    Descripcion TEXT,
    Precio DECIMAL(10,2) NOT NULL,
    Stock INT NOT NULL,
    CategoriaID INT REFERENCES Categorias(CategoriaID)
);

-- Tabla: Órdenes
CREATE TABLE Ordenes (
    OrdenID SERIAL PRIMARY KEY,
    UsuarioID INT REFERENCES Usuarios(UsuarioID),
    FechaOrden TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    Total DECIMAL(10,2) NOT NULL,
    Estado VARCHAR(50) DEFAULT 'Pendiente'
);

-- Tabla: Detalle de Órdenes
CREATE TABLE DetalleOrdenes (
    DetalleID SERIAL PRIMARY KEY,
    OrdenID INT REFERENCES Ordenes(OrdenID),
    ProductoID INT REFERENCES Productos(ProductoID),
    Cantidad INT NOT NULL,
    PrecioUnitario DECIMAL(10,2) NOT NULL
);

-- Tabla: Direcciones de Envío
CREATE TABLE DireccionesEnvio (
    DireccionID SERIAL PRIMARY KEY,
    UsuarioID INT REFERENCES Usuarios(UsuarioID),
    Calle VARCHAR(255) NOT NULL,
    Ciudad VARCHAR(100) NOT NULL,
    Departamento VARCHAR(100),
    Provincia VARCHAR(100),
    Distrito VARCHAR(100),
    Estado VARCHAR(100),
    CodigoPostal VARCHAR(20),
    Pais VARCHAR(100) NOT NULL
);

-- Tabla: Carrito de Compras
CREATE TABLE Carrito (
    CarritoID SERIAL PRIMARY KEY,
    UsuarioID INT REFERENCES Usuarios(UsuarioID),
    ProductoID INT REFERENCES Productos(ProductoID),
    Cantidad INT NOT NULL,
    FechaAgregado TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

-- Tabla: Métodos de Pago
CREATE TABLE MetodosPago (
    MetodoPagoID SERIAL PRIMARY KEY,
    Nombre VARCHAR(100) NOT NULL,
    Descripcion VARCHAR(255)
);

-- Tabla: Ordenes Métodos de Pago
CREATE TABLE OrdenesMetodosPago (
    OrdenMetodoID SERIAL PRIMARY KEY,
    OrdenID INT REFERENCES Ordenes(OrdenID),
    MetodoPagoID INT REFERENCES MetodosPago(MetodoPagoID),
    MontoPagado DECIMAL(10,2) NOT NULL
);

-- Tabla: Reseñas de Productos
CREATE TABLE ReseñasProductos (
    ReseñaID SERIAL PRIMARY KEY,
    UsuarioID INT REFERENCES Usuarios(UsuarioID),
    ProductoID INT REFERENCES Productos(ProductoID),
    Calificacion INT CHECK (Calificacion >= 1 AND Calificacion <= 5),
    Comentario TEXT,
    Fecha TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

-- Tabla: Historial de Pagos
CREATE TABLE HistorialPagos (
    PagoID SERIAL PRIMARY KEY,
    OrdenID INT REFERENCES Ordenes(OrdenID),
    MetodoPagoID INT REFERENCES MetodosPago(MetodoPagoID),
    Monto DECIMAL(10,2) NOT NULL,
    FechaPago TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    EstadoPago VARCHAR(50) DEFAULT 'Procesando'
);

Carga de CSV: a partir de esta seccion s ecargan los csv

Instalar dependencias en el entorno

In [3]:
%pip install pandas sqlalchemy psycopg2-binary python-dotenv

   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.1 MB ? eta -:--:--
   --------- ------------------------------ 0.5/2.1 MB 2.0 MB/s eta 0:00:01
   ------------------- -------------------- 1.0/2.1 MB 1.9 MB/s eta 0:00:01
   ------------------------ --------------- 1.3/2.1 MB 1.9 MB/s eta 0:00:01
   ---------------------------------- ----- 1.8/2.1 MB 2.0 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 1.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.6 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.6 MB 2.0 MB/s eta 0:00:02
   ---------------- ----------------------- 1.0/2.6 MB 1.7 MB/s eta 0:00:01
   -------------------- ------------------- 1.3/2.6 MB 1.8 MB/s eta 0:00:01
   ------------------------ --------------- 1.6/2.6 MB 1.8 MB/s eta 0:00:01
   -------------------------------- ----

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


Importar librerías

In [10]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Text, DECIMAL, TIMESTAMP, ForeignKey
from dotenv import load_dotenv
from sqlalchemy.orm import sessionmaker
import os

Conexion a la base de datos

In [11]:
load_dotenv()  # Carga las credenciales desde el archivo .env (seguridad)

# Variables para conectar con PostgreSQL
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
host = "localhost"
port = "5432"
database = "EcommerceDB"

# Crea el motor SQLAlchemy que sirve de puente entre Python y la BD
engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{database}")

# Base es la clase madre para definir modelos ORM que se conectan con tablas
Base = declarative_base()

# Crea una fábrica de sesiones que permite interactuar con la BD (insertar, consultar, etc.)
Session = sessionmaker(bind=engine)
session = Session()


C:\Users\valentin.e.bussolini\AppData\Local\Temp\ipykernel_34248\4226455634.py:14: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


Definir el modelo para cada una de las tablas

In [ ]:
# 🧍‍♂️ TABLA USUARIOS
class Usuario(Base):
    __tablename__ = 'usuarios'

    usuarioid = Column(Integer, primary_key=True)
    nombre = Column(String(100), nullable=False)
    apellido = Column(String(100), nullable=False)
    dni = Column(String(20), unique=True, nullable=False)
    email = Column(String(255), unique=True, nullable=False)
    contraseña = Column(String(255), nullable=False)
    fecharegistro = Column(TIMESTAMP)

# 🗂️ TABLA CATEGORÍAS
class Categoria(Base):
    __tablename__ = 'categorias'

    categoriaid = Column(Integer, primary_key=True)
    nombre = Column(String(100), nullable=False)
    descripcion = Column(String(255))

# 📦 TABLA PRODUCTOS
class Producto(Base):
    __tablename__ = 'productos'

    productoid = Column(Integer, primary_key=True)
    nombre = Column(String(255), nullable=False)
    descripcion = Column(Text)
    precio = Column(DECIMAL(10,2), nullable=False)
    stock = Column(Integer, nullable=False)
    categoriaid = Column(Integer, ForeignKey('categorias.categoriaid'))

# 🧾 TABLA ÓRDENES
class Orden(Base):
    __tablename__ = 'ordenes'

    ordenid = Column(Integer, primary_key=True)
    usuarioid = Column(Integer, ForeignKey('usuarios.usuarioid'))
    fechaorden = Column(TIMESTAMP)
    total = Column(DECIMAL(10,2), nullable=False)
    estado = Column(String(50))

# 📄 TABLA DETALLE DE ÓRDENES
class DetalleOrden(Base):
    __tablename__ = 'detalleordenes'

    detalleid = Column(Integer, primary_key=True)
    ordenid = Column(Integer, ForeignKey('ordenes.ordenid'))
    productoid = Column(Integer, ForeignKey('productos.productoid'))
    cantidad = Column(Integer, nullable=False)
    preciounitario = Column(DECIMAL(10,2), nullable=False)

# 📍 TABLA DIRECCIONES DE ENVÍO
class DireccionEnvio(Base):
    __tablename__ = 'direccionesenvio'

    direccionid = Column(Integer, primary_key=True)
    usuarioid = Column(Integer, ForeignKey('usuarios.usuarioid'))
    calle = Column(String(255), nullable=False)
    ciudad = Column(String(100), nullable=False)
    departamento = Column(String(100))
    provincia = Column(String(100))
    distrito = Column(String(100))
    estado = Column(String(100))
    codigopostal = Column(String(20))
    pais = Column(String(100), nullable=False)

# 🛒 TABLA CARRITO DE COMPRAS
class Carrito(Base):
    __tablename__ = 'carrito'

    carritoid = Column(Integer, primary_key=True)
    usuarioid = Column(Integer, ForeignKey('usuarios.usuarioid'))
    productoid = Column(Integer, ForeignKey('productos.productoid'))
    cantidad = Column(Integer, nullable=False)
    fechaagregado = Column(TIMESTAMP)

# 💳 TABLA MÉTODOS DE PAGO
class MetodoPago(Base):
    __tablename__ = 'metodospago'

    metodopagoid = Column(Integer, primary_key=True)
    nombre = Column(String(100), nullable=False)
    descripcion = Column(String(255))

# 🧾 TABLA ÓRDENES-MÉTODOS DE PAGO
class OrdenMetodoPago(Base):
    __tablename__ = 'ordenesmetodospago'

    ordenmetodoid = Column(Integer, primary_key=True)
    ordenid = Column(Integer, ForeignKey('ordenes.ordenid'))
    metodopagoid = Column(Integer, ForeignKey('metodospago.metodopagoid'))
    montopagado = Column(DECIMAL(10,2), nullable=False)

# 🌟 TABLA RESEÑAS DE PRODUCTOS
class ReseñaProducto(Base):
    __tablename__ = 'reseñasproductos'

    reseñaid = Column(Integer, primary_key=True)
    usuarioid = Column(Integer, ForeignKey('usuarios.usuarioid'))
    productoid = Column(Integer, ForeignKey('productos.productoid'))
    calificacion = Column(Integer)
    comentario = Column(Text)
    fecha = Column(TIMESTAMP)

# 🧾 TABLA HISTORIAL DE PAGOS
class HistorialPago(Base):
    __tablename__ = 'historialpagos'

    pagoid = Column(Integer, primary_key=True)
    ordenid = Column(Integer, ForeignKey('ordenes.ordenid'))
    metodopagoid = Column(Integer, ForeignKey('metodospago.metodopagoid'))
    monto = Column(DECIMAL(10,2), nullable=False)
    fechapago = Column(TIMESTAMP)
    estadopago = Column(String(50))

insertar datos en las tablas